# 00000: Model validation and optimization
Authors: Tobias G. Mueller, Mark A. Buckner
Last modified: 4 Dec 2024
Contact: __________

**Summary**: Here, we use the tiled model predictions as well as validate it against a labeled ground truth.
We then calculate the confidence threshold that optimizes f1 score and filter out main model predictions are that level.


This script outputs 
- an optimized model predicting on the whole orthomosaic
- model performance metrics

The data used in this script was generated in:
    `AIggregation/notebooks/02_predict_and_tile.ipynb`
    - labelstudio, labeling ground truth of the output from above script

Before running this script you must upload the tiled testset output from `02_predict_and_tile.ipynb` into label studio and export the output 

In [ ]:
#imports 
import os
import fiftyone as fo
from fiftyone import ViewField as F
import scipy
import fiftyone as fo

# first check the wd is not notebooks but the main folder
print("cwd is", os.getcwd())

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
    print("cwd changed to", os.getcwd())


In [ ]:
'''
import the test subset as fiftyone datasets

then merge files with different labels to view them all at once

'''

# -------------------------------------------------------------------------------------------------------------------- #
gt_directory = "datasets/testset/groundtruth_testset" # whereever labelstudio export in yolov5 format
tiled_test_directory = "datasets/testset/tiled_testset" # target folder for file script
# -------------------------------------------------------------------------------------------------------------------- #



# first rename label studio outputs to have same name as fiftyone outputs (label studio adds weird strings upon export)
for root, dirs, files in os.walk(gt_directory):
    for file in files:
        oldname = os.path.join(root,file)
        if "-" in file:
            newname = file.split("-")[1]
            os.rename(oldname, os.path.join(root,newname))



# import in fiftyone
datasettest = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = os.path.join(tiled_test_directory, "data.yaml"),
    label_field= "predictions"
)

dataset_ground = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = os.path.join(gt_directory,"data.yaml"),
    label_field= "ground_truth"
)

# merge the datasets, ignoring that they are in different directories and instead merging images/labels with the same file name

key_fcn = lambda sample: os.path.basename(sample.filepath)

datasettest.merge_samples(dataset_ground, key_fcn=key_fcn)



# delete any oblong boxes from the test dataset
# these are created by splitting tiles through the middle of an existing bounding box

# Computes the dimensions of each bounding box in pixels
box_width, box_height = F("bounding_box")[2], F("bounding_box")[3]

# get rid of detections where one side of box is greater than 2.5 x the other
datasettest_keep=datasettest.select_fields("predictions","ground_truth").filter_labels(
    "predictions", (box_height < (box_width*2)) & (box_width < (box_height*2)), only_matches=False
)

# create a subset of just boxes that were removed incase you want to visualize
removed_boxes=datasettest.select_fields("predictions").filter_labels(
    "predictions", (box_height > (box_width*2)) | (box_width > (box_height*2)) 
)

# view that this worked
session = fo.launch_app(datasettest_keep)



In [ ]:

# find confidence threshold that gives highest f1 value

# -------------------------------------------------------------------------------------------------------------------- #
dataset = datasettest_keep
prediction = "predictions"
gt = "ground_truth"
lb = .1
ub = .99
# -------------------------------------------------------------------------------------------------------------------- #




# function to calculate the f1 score of a model
def calculate_f1(conf, dataset, prediction, gt):
    conf_view = dataset.filter_labels(prediction, F("confidence") >= conf)
    results = conf_view.evaluate_detections(prediction,
        gt_field= gt,
        eval_key="eval",
        missing="fn")

    fp = sum(conf_view.values("eval_fp"))
    tp = sum(conf_view.values("eval_tp"))
    fn = sum(conf_view.values("eval_fn"))

    f1 = tp/(tp+0.5*(fp+fn))

    return -1.0*f1  #make output negative to use fminbound

# function find the optimal output of the above function
def optimize_conf(lb, ub, dataset, gt, prediction):

    res = scipy.optimize.fminbound(
                    func=calculate_f1,
                    x1=lb,
                    x2=ub,
                    args=(dataset, prediction, gt),
                    xtol=0.01,
                    full_output=True
    )

    best_conf, f1val, ierr, numfunc = res
    maxf1 = -1.0*f1val
    print("\n \n     best f1          at confidence")
    print(maxf1, best_conf)
    return maxf1, best_conf


# save first output, maxf1, to a variable and use it to filder predictions
bestconf = optimize_conf(lb=lb, ub=ub, dataset=dataset, gt=gt, prediction=prediction)[1]


# evaluate model performance at best conf threshold
high_f1_view = datasettest_keep.filter_labels("predictions", F("confidence") > bestconf, only_matches=False)

results_f1 = high_f1_view.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="predictions",
    compute_mAP=True
)

# print performance results
print(results_f1.mAP())
results_f1.print_report()


# create views for false positives and false negatives
fp_view = high_f1_view.to_evaluation_patches(eval_key="predictions").match(F("type")=="fp").sort_by("predictions.detection.confidence")
fn_view = high_f1_view.to_evaluation_patches(eval_key="predictions").match(F("type")=="fn").sort_by("predictions.detection.confidence")

# view false positives
#session.view = fp_view.view()

#view false negatives
#session.view = fn_view.view()



In [ ]:
# filter full predictions at same confidence threshold and export

# reimport full predictions

# -------------------------------------------------------------------------------------------------------------------- #
full_prediction_directory = "datasets/export_predictions/temp"
full_export_directory = "datasets/export_predictions/final"
# -------------------------------------------------------------------------------------------------------------------- #



# read in full sahi predictions
dataset_final = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = os.path.join(full_prediction_directory, "data.yaml"),
    label_field= "predictions"
)

# filter at optimal confidence
dataset_full_f1 = dataset_final.filter_labels("predictions", F("confidence") > bestf1, only_matches=False)

# export as final prediction
dataset_full_f1.export(
        export_dir=full_export_directory,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field="prediction",
        include_confidence=True
    )